Cómo Funciona
Modelo LLM:

Utilizamos un modelo de Hugging Face (distilbert-base-uncased) configurado para tareas de clasificación de texto.
Este modelo devuelve un puntaje de similitud entre el texto procesado (consulta + texto objetivo).
División en Palabras:

La consulta se divide en palabras para que SHAP pueda evaluar la importancia de cada una.
predict_text:

Genera texto filtrado basado en las palabras activas y calcula un puntaje de similitud con el modelo LLM.
Referencia SHAP:

Una matriz aleatoria de referencia ayuda a SHAP a generar explicaciones al comparar diferentes combinaciones de palabras.
Visualización:

Se utiliza shap.force_plot para mostrar cómo cada palabra contribuye al puntaje de similitud.

In [ ]:
import shap
import numpy as np
from transformers import pipeline


In [ ]:

# Modelo de lenguaje de Hugging Face
llm = pipeline("text-classification", model="distilbert-base-uncased", return_all_scores=True)

# Consulta y texto objetivo
query = "How does turmeric help in health?"
target_text = "Turmeric is beneficial for health because it has anti-inflammatory properties."

# Dividir la consulta en palabras
words = query.split()

# Función para la predicción con LLM
def predict_text(words_to_keep):
    """
    Genera predicciones para la consulta manteniendo solo ciertas palabras activas.
    """
    # Reconstruir el texto basado en las palabras seleccionadas
    filtered_query = " ".join([words[i] for i in range(len(words)) if words_to_keep[i] > 0.5])
    
    # Si no hay palabras, retorna un puntaje bajo por defecto
    if not filtered_query.strip():
        return [0.0]
    
    # Concatenar el texto filtrado con el texto objetivo
    input_text = f"{filtered_query} {target_text}"
    
    # Realizar predicción con el modelo
    prediction = llm(input_text)
    
    # Usar el puntaje de la clase positiva como salida
    return [prediction[0][0]['score']]

# Crear un conjunto de referencia inicial para SHAP
reference = np.random.rand(100, len(words))  # Matriz bidimensional con valores aleatorios

# Crear el explicador SHAP
explainer = shap.KernelExplainer(predict_text, reference)

# Calcular los valores SHAP para la consulta
shap_values = explainer.shap_values(np.ones(len(words)))

# Visualización de los valores SHAP
shap.force_plot(
    explainer.expected_value[0],
    shap_values[0],
    feature_names=words
)
